In [1]:
# %pip install kor

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 基础代码引入
from dotenv import load_dotenv
import os

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_base = os.getenv('GROQ_API_BASE')

print(groq_api_key, groq_api_base)

from langchain_openai import ChatOpenAI

# llama3-70b-8192
# llama3-8b-8192
# mixtral-8x7b-32768
# gemma-7b-it
# whisper-large-v3
# qwen 兼容 openai的接口
groq_llm_openai = ChatOpenAI(
    openai_api_base=groq_api_base,
    openai_api_key=groq_api_key,
    model_name="llama3-8b-8192",
    temperature=0.7,
    streaming=True,
)

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 https://groq-api.923828.xyz/v1


In [3]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

In [4]:
import json


def json_dump(json_object):
    json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False)
    print(json_formatted_str)

In [9]:
club_schema = Object(
    id="club",
    description="Information about a club",

    attributes=[
        Text(
            id="name",
            description="The name of the club."
        ),
        Text(
            id="manager",
            description="The name of the manager."
        ),
        Number(
            id="ranking",
            description="The ranking of the club."
        )
    ],
    examples=[
        (
            "Napoli managed by Spalletti are the 1st, Juventus managed by Allegri are the 5th.",
            [
                {"name": "Napoli", "manager": "Spalletti", "ranking": 1},
                {"name": "Juventus", "manager": "Allegri", "ranking": 5},
            ],
        )
    ]
)

In [7]:
# Create an extraction chain
extraction_chain = create_extraction_chain(groq_llm_openai, club_schema)

In [13]:
text = '''
    Manchester City are just 1 point after Arsenal. 
    Arsenal are on top of the table.
    The 2 managers Pep and Arteta are going to meet face to face in this weekend.
'''
output = extraction_chain.invoke(text)

In [14]:
json_dump(output)

{
  "text": {
    "data": {
      "club": [
        {
          "name": "Arsenal",
          "manager": "Arteta",
          "ranking": "1"
        },
        {
          "name": "Manchester City",
          "manager": "Pep",
          "ranking": "2"
        }
      ]
    },
    "raw": "name|manager|ranking\nArsenal|Arteta|1\nManchester City|Pep|2",
    "errors": [],
    "validated_data": {}
  }
}


In [15]:
stock_market_schema = Object(
    id="stock_market",
    description="A股市场收评，上涨板块综述",

    # Notice I put multiple fields to pull out different attributes
    attributes=[
        Text(
            id="industry_name",
            description="行业板块名称"
        )
    ],
    examples=[
        (
            '''
                周二市场呈现出震荡调整格局，盘中行业板块及题材呈现局部轮动和分化，保险、煤炭、通信设备、银行、家用电器、IT设备、有色、电气设备等行业活跃。
                酒店餐饮、半导体、旅游、多元金融、电器仪表、元器件、通用机械、商业连锁、医药等行业呈现回调。
            ''',
            [
                {"industry_name": "保险"},
                {"industry_name": "煤炭"},
                {"industry_name": "通信设备"},
                {"industry_name": "银行"},
                {"industry_name": "家用电器"},
                {"industry_name": "IT设备"},
                {"industry_name": "有色"},
                {"industry_name": "电气设备"}
            ],
        ),
        (
            '''
                截至2023年4月14日收盘，上证指数收涨0.60%，报收3338.15点;深证成指涨0.51%，报收11800.09点;创业板指涨0.93%，报收2428.09点。

                沪深两市全天成交额11150亿元，北向资金净买入64.24亿元，其中沪股通净买入21.58亿元，深股通净买入42.66亿元。

                根据申万二级行业分类，2023年4月14日共有66个行业上涨，其中，能源金属、半导体、电子化学品、工业金属、小金属涨幅居前;

                58个行业下跌，酒店餐饮、旅游及景区、广告营销、互联网电商、数字媒体跌幅居前。

                2023年4月14日，沪深A股共有2655家公司上涨，46家公司涨幅超10%(含)，华星创业、耐科装备、联动科技、拓荆科技、经纬辉开涨幅居前;

                有2102家公司下跌，4家公司跌幅超10%(含)，登康口腔、值得买、奥飞娱乐、慈文传媒、金桥信息跌幅居前。

            ''',
            [
                {"industry_name": "能源金属"},
                {"industry_name": "半导体"},
                {"industry_name": "电子化学品"},
                {"industry_name": "工业金属"},
                {"industry_name": "小金属"}
            ],
        ),
        (
            '''
                传媒、游戏、数据确权、贵金属等板块涨幅居前，工业母机、白酒股等板块飘绿。
            ''',
            [
                {"industry_name": "传媒"},
                {"industry_name": "游戏"},
                {"industry_name": "数字确权"},
                {"industry_name": "贵金属"}
            ],
        )
    ]
)

In [16]:
text = '''
    截至2023年4月13日收盘，上证指数收跌0.27%，报收3318.36点;深证成指跌1.21%，报收11739.84点;创业板指跌0.97%，报收2405.76点。

    沪深两市全天成交额11369亿元，北向资金净卖出8688万元，其中沪股通净买入20.64亿元，深股通净卖出21.51亿元。

    根据申万二级行业分类，2023年4月13日共有46个行业上涨，其中，旅游及景区、酒店餐饮、出版、教育、互联网电商涨幅居前;

    78个行业下跌，通信设备、半导体、元件、消费电子、黑色家电跌幅居前。
'''
extraction_chain = create_extraction_chain(groq_llm_openai, stock_market_schema)
output = extraction_chain.invoke(text)

In [17]:
json_dump(output)

{
  "text": {
    "data": {
      "stock_market": [
        {
          "industry_name": "旅游及景区"
        },
        {
          "industry_name": "酒店餐饮"
        },
        {
          "industry_name": "出版"
        },
        {
          "industry_name": "教育"
        },
        {
          "industry_name": "互联网电商"
        }
      ]
    },
    "raw": "industry_name\n旅游及景区\n酒店餐饮\n出版\n教育\n互联网电商",
    "errors": [],
    "validated_data": {}
  }
}
